In [3]:
from pd4ml import Airshower

In [4]:
u = Airshower.load_data()

/home/djanloo/.local/share/virtualenvs/cmepda-VJ-vmc_S/lib/python3.9/site-packages/rich/live.
py:221: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')